# Reconocimiento de pájaros por su canto! (Preámbulo)

**Especies consideradas**:

    - Gorrion comun      (Passer domesticus)
    - Mirlo comun        (Turdus merula)
    - Petirrojo europeo  (Erithacus Rubecula)
    - Jilguero europeo   (Carduelis carduelis)
    
    Próximamente:
    - Canario español  (Serinus canaria)
    - Golondrina común  (Hirundo rustica)
    - Urraca  (pica pica)


## Descarga de los datos

- [x] Implementar que solo se descarguen .wav
- [x] Si no hay suficientes .wav descargar .mp3
- [ ] Iterar bajando el bitrate de los mp3 o comprobar si hay duración suficiente

In [1]:
#!pip install pydub
#!pip install mutagen

import requests
import os
from pydub import AudioSegment  # Para verificar el bitrate de MP3s
from io import BytesIO
from mutagen.mp3 import MP3

In [ ]:
def get_total_duration(scientific_name, quality="A"):
    """
    Obtiene la duración total disponible de grabaciones para una especie desde Xeno-Canto.
    
    Args:
        scientific_name (str): Nombre científico de la especie (e.g., "Passer domesticus").
        quality (str): Calidad mínima de las grabaciones a considerar (e.g., "A" o "B").
    
    Returns:
        total_duration (int): Duración total en segundos de las grabaciones disponibles.
        num_recordings (int): Número de grabaciones disponibles.
    """
    base_url = f"https://www.xeno-canto.org/api/2/recordings?query={scientific_name.replace(' ', '%20')}+q:{quality}"
    total_duration = 0
    num_recordings = 0
    page = 1  # La API usa paginación
    
    while True:
        #print(f"Consultando página {page} para {scientific_name}...")
        response = requests.get(base_url + f"&page={page}")
        if response.status_code != 200:
            print(f"Error al consultar la API. Código: {response.status_code}")
            break
        
        data = response.json()
        recordings = data.get("recordings", [])
        
        for rec in recordings:
            # Convertir 'mm:ss' a segundos
            length = rec.get("length", "0:00")  # Longitud en formato 'mm:ss'
            minutes, seconds = map(int, length.split(":"))  # Dividir y convertir a entero
            duration = minutes * 60 + seconds  # Convertir a segundos
            
            total_duration += duration
            num_recordings += 1
        
        # Comprobar si hay más páginas
        if not data.get("numPages") or page >= int(data["numPages"]):
            break
        page += 1
    
    return total_duration, num_recordings

In [ ]:
def get_total_filetype_duration(scientific_name, filetype, quality="A"):
    """
    Obtiene la duración total disponible de grabaciones para una especie desde Xeno-Canto.
    
    Args:
        scientific_name (str): Nombre científico de la especie (e.g., "Passer domesticus").
        quality (str): Calidad mínima de las grabaciones a considerar (e.g., "A" o "B").
    
    Returns:
        total_duration (int): Duración total en segundos de las grabaciones disponibles.
        num_recordings (int): Número de grabaciones disponibles.
    """
    base_url = f"https://www.xeno-canto.org/api/2/recordings?query={scientific_name.replace(' ', '%20')}+q:{quality}"
    total_duration = 0
    num_recordings = 0
    page = 1  # La API usa paginación
    
    while True:
        print(f"Consultando página {page} para {scientific_name}...")
        response = requests.get(base_url + f"&page={page}")
        if response.status_code != 200:
            print(f"Error al consultar la API. Código: {response.status_code}")
            break
        
        data = response.json()
        recordings = data.get("recordings", [])
        
        for rec in recordings:
            # Construir URL de archivo
            file_url = rec.get('file', '')
            
            audio_response = requests.get(file_url, stream=True)
            content_type = audio_response.headers.get('Content-Type', '')
            if 'audio/wav' in content_type:
                # Convertir 'mm:ss' a segundos
                length = rec.get("length", "0:00")  # Longitud en formato 'mm:ss'
                minutes, seconds = map(int, length.split(":"))  # Dividir y convertir a entero
                duration = minutes * 60 + seconds  # Convertir a segundos
                
                total_duration += duration
                num_recordings += 1
        
        # Comprobar si hay más páginas
        if not data.get("numPages") or page >= int(data["numPages"]):
            break
        page += 1
    
    return total_duration, num_recordings

In [ ]:
species_list = {"Passer domesticus",
                "Turdus merula",
                "Erithacus Rubecula",
                "Carduelis carduelis"}

In [ ]:
for species in species_list:
    duration, num_recordings = get_total_filetype_duration(species, "wav", quality="A")
    print(f"Duración total disponible para {species}: {duration / 3600:.2f} horas")
    print(f"Número de grabaciones disponibles: {num_recordings}")
    print('\n')

In [3]:
def download_species_duration(scientific_name, output_folder, target_duration=1800, quality="A"):
    """
    Descarga grabaciones de Xeno-Canto en formato .wav para una especie específica y calcula la duración total.
    
    Args:
        scientific_name (str): Nombre científico de la especie (e.g., "Passer domesticus").
        output_folder (str): Carpeta para guardar las grabaciones.
        target_duration (int): Duración total objetivo en segundos.
        quality (str): Calidad mínima de las grabaciones (e.g., "A" o "B").
    """
    os.makedirs(output_folder, exist_ok=True)  # Crear la carpeta de salida si no existe
    base_url = f"https://www.xeno-canto.org/api/2/recordings?query={scientific_name.replace(' ', '%20')}+q:{quality}"
    total_duration = 0
    num_recordings = 0
    page = 1

    # Crear la carpeta de la especie
    species_folder = os.path.join(output_folder, scientific_name.replace(' ', '_'), "Audios")
    os.makedirs(species_folder, exist_ok=True)

    while total_duration < target_duration:  # Iterar mientras no alcancemos la duración objetivo
        response = requests.get(base_url + f"&page={page}")
        if response.status_code != 200:
            print(f"Error al consultar la API. Código: {response.status_code}")
            break

        data = response.json()
        recordings = data.get("recordings", [])

        for rec in recordings:
            if total_duration >= target_duration:
                break
            
            # Construir URL de archivo
            file_url = rec.get('file', '')

            try:
                # Intentar descargar el archivo
                audio_response = requests.get(file_url, stream=True)
                content_type = audio_response.headers.get('Content-Type', '')

                # Filtrar solo archivos WAV (audio/wav) y MP3 (audio/mp3)
                if 'audio/wav' in content_type:
                    # Obtener la duración de la grabación en segundos
                    length = rec.get("length", "0:00")
                    minutes, seconds = map(int, length.split(":"))
                    duration = minutes * 60 + seconds

                    # Generar un nombre único para el archivo (e.g., 'gorrion_1.wav', 'gorrion_2.wav', etc.)
                    new_file_name = f"download_{num_recordings + 1}.wav"
                    file_path = os.path.join(species_folder, new_file_name)
                    
                    # Descargar el archivo###################################################################
                    with open(file_path, "wb") as f:
                        f.write(audio_response.content)
                    
                    total_duration += duration
                    num_recordings += 1
                    print(f"Descargado: .wav | Duración acumulada: {total_duration} segundos")
                    
                elif 'audio/mpeg' in content_type:
                    audio_data = BytesIO(audio_response.content)
                    audio = AudioSegment.from_file(audio_data, format="mp3")
                    
                    # Obtener el bitrate en kbps -> Mutagen
                    mp3_audio = MP3(audio_data) 
                    bitrate_kbps = mp3_audio.info.bitrate // 1000

                    if bitrate_kbps >= 256:  # 256 kbps esta bastante bien (320 es increible)
                        new_file_name = f"download_{num_recordings + 1}.mp3"
                        file_path = os.path.join(species_folder, new_file_name)
                    
                        # Duracion grabacion en segundos
                        length = rec.get("length", "0:00")
                        minutes, seconds = map(int, length.split(":"))
                        duration = minutes * 60 + seconds
                        
                        # Descargar el archivo###################################################################
                        with open(file_path, "wb") as f:
                            f.write(audio_response.content)

                        total_duration += duration
                        num_recordings += 1
                        #print(f"Se ha descargado un .mp3 de bitrate: {bitrate_kbps}")
                        print(f"Descargado: .mp3 de bitrate: {bitrate_kbps} | Duración acumulada: {total_duration} segundos")

            except Exception as e:
                print(f"Error al descargar {file_url}: {e}")

        # Comprobar si hay más páginas
        if not data.get("numPages") or page >= int(data["numPages"]):
            break
        page += 1

    print(f"Duración total descargada para {scientific_name}: {total_duration} segundos.")
    print(f"Número total de grabaciones descargadas: {num_recordings}")

In [ ]:
# Ejemplo de uso
download_species_duration("Passer domesticus", "C:/pajarillos", target_duration=1000, quality="A")
#for species in species_list:
#    download_species_duration(species, "D:/pajarillos", target_duration=100, quality="A")

## Creacion de espectrogramas y de la base de datos

- Preprocesamiento de los datos
    - [x] Cortar en fragmentos de 5 s
    - [x] Convertir a espectrogramas Mel
    - [x] Resize: 300x300 y [0,255]
    - [x] Dividir en **train**, **val** y **test**
        - [ ] Nota: considerar la importancia de un diferente número de espectrogramas: ~100 
    - [ ] Mejorar comentarios

In [3]:
#!pip install librosa
#!pip install opencv-python

import librosa
import librosa.display
import numpy as np
import os
import matplotlib.pyplot as plt
import soundfile as sf
import cv2

import os
import shutil
import random

In [5]:
# Parámetros
segment_duration = 5  # Duración del fragmento en segundos
sr_target = 44100  # Frecuencia de muestreo en Hz (44.1 kHz)
n_mels = 128  # Número de bandas Mel
n_fft = 1024  # Tamaño de la ventana FFT
hop_length = n_fft // 4  # Desplazamiento entre ventanas (75% de superposición) (default, no sé muy bien el efecto)

In [7]:
def generate_mel_spectrogram(y, sr, output_filename):
    """Genera y guarda un espectrograma de Mel"""
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)  # Convertir a escala logarítmica (dB)

    # Normalizar a [0,255]
    mel_spec_db -= mel_spec_db.max() # En oscuro lo más intenso
    mel_spec_db /= mel_spec_db.min()
    mel_spec_db *= 255
    mel_spec_db = mel_spec_db.astype(np.uint8)

    # Reescalar a 300x300
    resized_spec = cv2.resize(mel_spec_db, (300, 300))

    # Guardar imagen en escala de grises
    cv2.imwrite(output_filename, resized_spec)

In [75]:
def process_audio_and_split(input_folder, output_folder, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, segment_duration=5):
    """Procesa archivos de audio y genera espectrogramas directamente en train, val y test"""
    splits = {"train": train_ratio, "val": val_ratio, "test": test_ratio}
    
    # Crear carpetas de destino
    for split in splits.keys():
        for species in os.listdir(input_folder):
            new_dirs = os.path.join(output_folder, split, species)
            os.makedirs(new_dirs, exist_ok=True)
        
    # Bucle con todas las especies en el directorio
    for species in os.listdir(input_folder):
        
        auds_folder = os.path.join(input_folder, species, "Audios")
        
        if os.path.isdir(auds_folder):  # Asegurar que es una carpeta
            print(f"Se accedió a {auds_folder}")
            all_auds = [] # Distribuir de manera uniforme cada especie
            
            for filename in os.listdir(auds_folder):
                if filename.endswith(".wav") or filename.endswith(".mp3"):
                    file_path = os.path.join(auds_folder, filename)
                    y, sr = librosa.load(file_path, sr=44100)
                    total_duration = librosa.get_duration(y=y, sr=sr)
                    num_segments = int(total_duration // segment_duration)
                    
                    for i in range(num_segments):
                        start_sample = i * segment_duration * sr
                        end_sample = start_sample + (segment_duration * sr)
                        
                        if end_sample <= len(y):
                            segment = y[start_sample:end_sample]
                            aud_filename = f"{os.path.splitext(filename)[0]}_seg_{i}.png"
                            all_auds.append((segment, sr, aud_filename))
            
            # Mezclar aleatoriamente
            random.shuffle(all_auds)
            
            num_total = len(all_auds)
            num_train = int(num_total * train_ratio)
            num_val = int(num_total * val_ratio)
            
            train_data = all_auds[:num_train]
            val_data = all_auds[num_train:num_train + num_val]
            test_data = all_auds[num_train + num_val:]
        else:
            print(f"No se pudo acceder a {auds_folder}")
            
        # Guardar en las carpetas correspondientes
        for split, data in zip(["train", "val", "test"], [train_data, val_data, test_data]):
            for segment, sr, filename in data:
                output_path = os.path.join(output_folder, split, species, filename)
                generate_mel_spectrogram(segment, sr, output_path)
    
    print("Espectrogramas generados y distribuidos en train/val/test.")

In [69]:
process_audio_and_split(input_folder="C:/pajarillos",
                        output_folder="C:/Dataset",
                        train_ratio=0.7,
                        val_ratio=0.2,
                        test_ratio=0.1,
                        segment_duration=5)

Passer_domesticus
C:/pajarillos\Passer_domesticus\Audios
Se accedió a C:/pajarillos\Passer_domesticus\Audios
Espectrogramas generados y distribuidos en train/val/test.
